# **우리말샘 API 끝말잇기(상대평가)**
링크 : ```https://opendict.korean.go.kr/main```

- **게임 방법**
    1. 가장 처음 단어를 제시하면 끝말잇기가 시작됩니다
    2. '/그만'을 입력하면 게임이 종료되며, '/다시'를 입력하여 게임을 다시 시작할 수 있습니다.

- **게임 기본 구현 규칙(40점)**
    1. 우리말샘 API에 등재된 명사여야 합니다.
    2. 단어의 길이가 두 글자 이상이어야 합니다.
    3. 이미 사용한 단어를 다시 사용할 수 없습니다.

- **본인만의 새로운 규칙을 추가(최대 20점, 상대평가)**
    1. 아래에 추가된 규칙을 작성 해야 합니다.
    2. 규칙은  함수 형태로 작동해야 합니다.
    3. 추가된 규칙의 구현난도, 창의성, 개수를 가지고 상대평가를 진행합니다.

- **추가된 규칙**
    1. 입력한 값이 명사인 경우에 입력한 단어의 의미를 설명
    2. /혼자하기 입력시 혼자서 계속해서 진행하지만, /대결하기를 입력하면 자동으로 일치하는 단어를 출력하여 대결 진행

In [31]:
import requests as request

# 명사 확인 기능 / 입력한 단어가 명사인 경우, 의미를 반환
def checkPos(sense, keyword):
  if sense['pos'] == '명사':
    return sense["definition"]

  else:
    return False

# 대결하기 기능 / 사용자가 입력한 단어의 마지막 글자로 시작하는 단어 중 사용하지 않은 단어와 의미를 반환
def fightAI(last_char, answer_set):
  answer_array = []
  response_result = checkAPI(last_char + '-')

  if int(response_result['channel']['total']) > 0:
    for item in response_result['channel']['item']:
      for sense in item['sense']:
        if checkPos(sense, item['word']):
          answer_array.append({'word': item['word'].replace('-', ''), 'definition': sense['definition']})

  for answer in answer_array:
    if answer['word'] not in answer_set:
      return answer

# 입력 문자 확인 기능 / 사용자가 입력한 문자가 이전 문자의 마지막 글자와 일치하는지, 2글자 이상인지, 이미 사용한 글자인지 확인 후 결과 반환
def checkKeyWord(keyword, answer_set):
  if last_char != '' and keyword[0] != last_char:
    print(f'▶ {last_char}로 시작하는 단어가 아닙니다. 다른 단어를 입력해주세요')
    return False

  elif len(keyword) <= 1:
    print('▶ 두 글자 이하의 단어를 입력할 수 없습니다. 다른 단어를 입력해주세요')
    return False

  elif keyword in answer_set:
    print('▶ 이미 사용된 단어입니다. 다른 단어를 입력해주세요')
    return False

  return True

def checkAPI(key_word):
  api_key = '886CA88AF1D14B0E3DB32D52DAEFAD9A'
  url = 'https://opendict.korean.go.kr/api/search'

  response = request.get(url, params={'key': api_key, 'q': key_word , 'req_type': 'json'})
  response_result = response.json()

  return response_result

answer_set = set()
last_word = ''
last_char = ''
gameOn = False
isFightAI = False

while True:
  print('\n▶ 사용 가능한 기능: /그만, /다시, /혼자하기, /대결하기')
  user_input = input('▶ 원하시는 기능을 입력해주세요: ')

  if user_input == '/혼자하기' or user_input == '/대결하기' or user_input == '/다시':
    gameOn = True

  if user_input == '/대결하기':
    isFightAI = True

  while gameOn:
    word = '' if last_char == '' else last_char + '로 시작하는 ';
    keyword = input(f'\n▶ {word}단어를 입력해주세요(/그만을 입력하면 게임이 종료됩니다): ')

    if keyword == '/그만':
      print('▶ 게임을 종료합니다')
      gameOn = False
      isFightAI = False
      break

    if checkKeyWord(keyword, answer_set) == False:
      continue

    response_result = checkAPI(keyword)
    response_result_count = int(response_result['channel']['total'])

    if response_result_count > 0:
      checkPosResult = checkPos(response_result['channel']['item'][0]['sense'][0], keyword)

      if checkPosResult != False:
        print(f'▶ 사용 가능한 단어입니다: {keyword} / {checkPosResult}')
        last_word = last_word + ' - ' if last_word != '' else ''
        print(f'▶ {last_word}{keyword}')

        answer_set.add(keyword)
        last_char = keyword[-1]
        last_word = keyword

        if isFightAI:
          ai_answers = fightAI(last_char, answer_set)

          word = ai_answers["word"]

          print(f'\n▶ AI의 답변: {word} / {ai_answers["definition"]}')
          print(f'▶ {keyword} - {word}')

          answer_set.add(word)
          last_char = word[-1]
          last_word = word
      else:
        print('▶ 명사가 아닙니다. 다른 단어를 입력해주세요')
    else:
      print('▶ 존재하지 않는 단어입니다. 다른 단어를 입력해주세요')



▶ 사용 가능한 기능: /그만, /다시, /혼자하기, /대결하기
▶ 원하시는 기능을 입력해주세요: /대결하기

▶ 단어를 입력해주세요(/그만을 입력하면 게임이 종료됩니다): 나무
▶ 사용 가능한 단어입니다: 나무 / 땔감이 되는 나무붙이.
▶ 나무

▶ AI의 답변: 무가사 / 가사가 없음.
▶ 나무 - 무가사


KeyboardInterrupt: Interrupted by user